In [1]:
print("hey")

hey


In [9]:
import os
os.chdir("../")

In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [12]:
extracted_data = load_pdf_file('data')
type(extracted_data[0])

langchain_core.documents.base.Document

In [15]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=30)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [16]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5659

In [17]:
text_chunks

[Document(metadata={'source': 'data/K4x8a2_Sports_and_Exercise_Nutrition-_3rd_edition.pdf', 'page': 1}, page_content='98421_FM  2/15/08  7:57 AM  Page ii'),
 Document(metadata={'source': 'data/K4x8a2_Sports_and_Exercise_Nutrition-_3rd_edition.pdf', 'page': 2}, page_content='98421_FM  2/15/08  7:57 AM  Page i'),
 Document(metadata={'source': 'data/K4x8a2_Sports_and_Exercise_Nutrition-_3rd_edition.pdf', 'page': 3}, page_content='98421_FM  2/15/08  7:57 AM  Page ii'),
 Document(metadata={'source': 'data/K4x8a2_Sports_and_Exercise_Nutrition-_3rd_edition.pdf', 'page': 4}, page_content='WILLIAM D. McARDLE\nProfessor Emeritus\nDepartment of Family, Nutrition, and Exercise Science\nQueens College of the City University of New York\nFlushing, New York\nFRANK I. KATCH\nInternational Research Scholar\nFaculty of Public Health, Sport, and Nutrition\nAgder University College\nKristiansand, Norway\nInstructor and Board Member\nCertificate Program in Fitness Instruction\nUniversity of California at L

In [29]:
from langchain.embeddings import OpenAIEmbeddings

In [30]:
#Download the Embeddings from Hugging Face
def AIEMBEDDINGS():
    embeddings = OpenAIEmbeddings()
    return embeddings

In [31]:
embeddings = AIEMBEDDINGS()

/var/folders/c3/4kvxr4c12w9dnl5w6360qn0c0000gn/T/ipykernel_95013/123700136.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [32]:
query_result = embeddings.embed_query("Hello There")
print(len(query_result))

1536


In [33]:
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [34]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "nutribot"

pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [35]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [36]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [37]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [39]:
retrieved_docs = retriever.invoke("What is a good diet?")
retrieved_docs

[Document(id='bc9675c0-9128-4287-b008-fe5d9de1fb47', metadata={'page': 258.0, 'source': 'data/K4x8a2_Sports_and_Exercise_Nutrition-_3rd_edition.pdf'}, page_content='www.my.webmd.com/content/tools/ 1/calcdessert.htm?2–\n36288100000000704\nBuild a Healthy Base\n1. Let MyPyramid guide your food choices. www.nal.\nusda.gov/fnic/Fpyr/pymid.gif\n2. Choose diverse grains on a daily basis, particularly\nunrefined, whole grains. www.cspinet.org/nah/wwheat.\nhtml\n3. Choose a variety of fruits and vegetables daily.\n4. Keep food safe to eat. www.fightbac.org/main.cfm\nChoose Sensibly\n1. Choose a diet low in saturated fat and cholesterol'),
 Document(id='b208c8fa-0f74-4b4e-9c2a-9a7bfab6564f', metadata={'page': 229.0, 'source': 'data/K4x8a2_Sports_and_Exercise_Nutrition-_3rd_edition.pdf'}, page_content='needs\n• Increase daily intake of fruits, vegetables, whole\ngrains, and nonfat or low-fat milk and milk prod-\nucts\n• Choose fats wisely for good health. Limit saturated\nfats (dairy, fatty meat

In [40]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.5,max_tokens=500)

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. Be friendly"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response = rag_chain.invoke({"input": "what is a good diet?"})
print(response["answer"])



A good diet includes a daily intake of fruits, vegetables, whole grains, and low-fat dairy products. It also involves choosing healthy fats and carbohydrates, such as monounsaturated fats and fiber-rich foods. It is important to follow MyPyramid guidelines and choose a variety of foods to maintain a balanced and nutritious diet.
